# Modelo de Negócio

    Eduardo e Marcelo são dois brasileiros, amigos e sócios de empreendimento. Depois de vários
    negócio bem sucedidos, eles estão planejando entrar no mercado de moda dos USA como um
    modelo de negócio do tipo E-commerce.
    A idéia inicial é entrar no mercado com apenas um produto e para um público específico, no caso
    o produto seria calças Jenas para o público masculino. O objetivo é manter o custo de operação
    baixo e escalar a medida que forem conseguindo clientes.
    Porém, mesmo com o produto de entrada e a audiência definidos, os dois sócios não tem experiência
    nesse mercado de moda e portanto não sabem definir coisas básicas como preço, o tipo de calça e
    o material para a fabricação de cada peça.
    Assim, os dois sócios contrataram uma consultoria de Ciência de Dados para responder as seguintes
    perguntas: 1. Qual o melhor preço de venda para as calças? 2. Quantos tipos de calças e suas
    cores para o produto inicial? 3. Quais as matérias-prima necessárias para confeccionar as calças?
    As principais concorrentes da empresa Start Jeans são as americadas H&M e Macys.

### 3.0.2 2.0 O método SAPE
    1. Qual o melhor preço de venda para as calças?
    2. Quantos tipos de calças e suas cores para o produto inicial?
    3. Quais as matérias-prima necessárias para confeccionar as calças?
### Saída ( o produto final)
    1. Resposta para a pergunta
    - A mediana dos valores dos produtos do site dos concorrentes.
    2. Formato
    - Tabela ou gráfico
    3. Local de entrega
    - App no Streamlit
### Processo ( passo a passo )
    1. Passo a passo para calcular a resposta?
    - Mediana do preço por categoria e tipo.
    2. Como será o gráfico ou tabela final?
    - Simulação da tabela final
    3. Como será o local de entrega?
    - Dashboard em um app no Streamlit e publicá-lo no Heroku.
### Entradas ( fontes de dados )
    1. H&M: https://www2.hm.com/en_us/men/products/jeans.html
    2. Macys: https://www.macys.com/shop/mens-clothing/mens-jeans

# IMPORTS

In [1]:
import requests
import sqlite3
import re
import pandas     as pd
import numpy      as np


from   datetime   import datetime
from   bs4        import BeautifulSoup
from   sqlalchemy import create_engine

In [2]:
# =================== # =================

In [3]:
#API Requests
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

page = requests.get( url, headers=headers )

In [4]:
soup = BeautifulSoup( page.text, 'html.parser' )

In [5]:
products = soup.find( 'ul', class_='products-listing small' )

In [6]:
product_list = products.find_all( 'article', class_='hm-product-item')
# product id
product_id = [p.get( 'data-articlecode' ) for p in product_list]
# product category
product_category = [p.get( 'data-category' ) for p in product_list]

In [7]:
# product name
product_list = products.find_all( 'a', class_='link' )
product_name = [p.get_text() for p in product_list]

In [8]:
# price
product_list = products.find_all( 'span', class_='price regular' )
product_price = [p.get_text() for p in product_list]

In [9]:
data = pd.DataFrame( [product_id, product_category, product_name, product_price] ).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']
# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )

# Multiple Products

In [10]:
# empty dataframe
df_details = pd.DataFrame()

# unique columns for all products
aux = []

cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size']
df_pattern = pd.DataFrame( columns=cols )

for i in range( len( data ) ):
    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + str(data.loc[i, 'product_id']) + '.html'
    page = requests.get( url, headers=headers )
    # Beautiful Soup object
    soup = BeautifulSoup( page.text, 'html.parser' )
    # ==================== color name =================================
    product_list = soup.find_all( 'a', class_='filter-option miniature' )
    color_name = [p.get( 'data-color' ) for p in product_list]
    # product id
    product_id = [p.get( 'data-articlecode' ) for p in product_list]
    df_color = pd.DataFrame( [product_id, color_name] ).T
    df_color.columns = ['product_id', 'color_name']
    # generate style id + color id
    df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
    df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )
    
    # ==================== composition =================================
    product_composition_list = soup.find_all( 'div', class_='pdp-description-list-item' )

    product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]

    # reaname dataframe
    df_composition = pd.DataFrame( product_composition ).T
    df_composition.columns = df_composition.iloc[0]

    # delete first row
    df_composition = df_composition.iloc[1:].fillna( method='ffill' )

    # garantee the same number of columns
    df_composition = pd.concat( [df_pattern, df_composition], axis=0 )

    # generate style id + color id
    df_composition['style_id'] = df_composition['Art. No.'].apply( lambda x: x[:-3] )
    df_composition['color_id'] = df_composition['Art. No.'].apply( lambda x: x[-3:] )
    aux = aux + df_composition.columns.tolist()

    # merge data color + decomposition
    data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit', 'Composition', 'Size', 'Product safety']], how='left', on='style_id' )

    # all details products
    df_details = pd.concat( [df_details, data_sku], axis=0 )
# Join Showroom data + details
data['style_id'] = data['product_id'].apply( lambda x: x[:-3] )
data['color_id'] = data['product_id'].apply( lambda x: x[-3:] )
data_raw = pd.merge( data, df_details[['style_id', 'color_name', 'Fit', 'Composition', 'Size', 'Product safety']], how='left', on='style_id' )

data_raw.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,Fit,Composition,Size,Product safety
0,0636207011,,Slim Jeans,$ 19.99,2021-08-31 20:54:18,0636207,011,Dark denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,NaN
1,0636207011,,Slim Jeans,$ 19.99,2021-08-31 20:54:18,0636207,011,Dark denim blue,Slim fit,"Cotton 89%, Polyester 10%, Elastane 1%",NaN,NaN
2,0636207011,,Slim Jeans,$ 19.99,2021-08-31 20:54:18,0636207,011,Dark gray denim,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,NaN
3,0636207011,,Slim Jeans,$ 19.99,2021-08-31 20:54:18,0636207,011,Dark gray denim,Slim fit,"Cotton 89%, Polyester 10%, Elastane 1%",NaN,NaN
4,0636207011,,Slim Jeans,$ 19.99,2021-08-31 20:54:18,0636207,011,Denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,NaN


In [11]:
# ================ Type change ===================== #

data = data_raw.copy()

# product id
data = data.dropna( subset=['product_id'] )
data['product_id'] = data['product_id'].astype( int )

# product name
data['product_name'] = data['product_name'].apply( lambda x: x.replace( ' ', '_' ).lower() )

# product price
data['product_price'] = data['product_price'].apply( lambda x: x.replace( '$ ', '' ) ).astype( float )

# scrapy datetime
data['scrapy_datetime'] = pd.to_datetime( data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S' )

## style id
data['style_id'] = data['style_id'].astype( int )

## color id
data['color_id'] = data['color_id'].astype( int )

# color name
data['color_name'] = data['color_name'].apply( lambda x: x.replace( ' ', '_' ).replace( '/', '_' ).lower() if pd.notnull( x ) else x )

## fit
data['Fit'] = data['Fit'].apply( lambda x: x.replace( ' ', '_' ).lower() if pd.notnull( x ) else x )
data.columns = data.columns.str.replace('Fit','fit')

# size number
data['size_number'] = data['Size'].apply( lambda x: re.search( '\d{3}cm', x ).group(0) if pd.notnull( x ) else x )
data['size_number'] = data['size_number'].apply( lambda x: re.search( '\d+', x).group(0) if pd.notnull( x ) else x )

# size model
data['size_model'] = data['Size'].str.extract( '(\d+/\\d+)' )
data = data.drop( columns=['Size', 'Product safety'], axis=1 )

# composition
data = data[~data['Composition'].str.contains( 'Pocket lining:', na=False )]
data = data[~data['Composition'].str.contains( 'Lining:', na=False )]
data = data[~data['Composition'].str.contains( 'Shell:', na=False )]

# drop duplicates
data = data.drop_duplicates( subset=['product_id', 'product_category', 'product_name', 'product_price', 
                                     'scrapy_datetime', 'style_id', 'color_id', 'color_name', 'fit'], keep='last' )


# reset index
data = data.reset_index( drop=True )

# break composition by comma
df1 = data['Composition'].str.split( ',', expand=True )

# cotton | polyester | elastano | elasterell
df_ref = pd.DataFrame( index=np.arange( len( data ) ), columns=['cotton','polyester', 'elastane', 'elasterell'] )

# cotton
df_cotton = df1[0]
df_cotton.name = 'cotton'

df_ref = pd.concat( [df_ref, df_cotton ], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]
df_ref['cotton'] = df_ref['cotton'].fillna( 'Cotton 0%' )

# polyester
df_polyester = df1.loc[df1[1].str.contains( 'Polyester', na=True ), 1]
df_polyester.name = 'polyester'
df_ref = pd.concat( [df_ref, df_polyester], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]
df_ref['polyester'] = df_ref['polyester'].fillna( 'Polyester 0%' )

# elastano
df_elastane = df1.loc[df1[1].str.contains( 'Elastane', na=True ), 1]
df_elastane.name = 'elastane'

# combine elastane from both columns 1 and 2
df_elastane = df_elastane.combine_first( df1[2] )
df_ref = pd.concat( [df_ref, df_elastane], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]
df_ref['elastane'] = df_ref['elastane'].fillna( 'Elastane 0%' )

# elasterell
df_elasterell = df1.loc[df1[1].str.contains( 'Elasterell', na=True ), 1]
df_elasterell.name = 'elasterell'
df_ref = pd.concat( [df_ref, df_elasterell], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]

df_ref['elasterell'] = df_ref['elasterell'].fillna( 'Elasterell-P 0%' )

# final join
data = pd.concat( [data, df_ref], axis=1 )

# format composition data
data['cotton'] = data['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
data['polyester'] = data['polyester'].apply( lambda x: int( re.search( '\d+', x).group(0) ) / 100 if pd.notnull( x ) else x )
data['elastane'] = data['elastane'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
data['elasterell'] = data['elasterell'].apply( lambda x: int( re.search('\d+',x ).group(0) ) / 100 if pd.notnull( x ) else x )

# Drop columns
#data = data.drop( columns=['size', 'product safety', 'composition'], axis=1 )
data = data.drop( 'Composition', axis=1 )

# Drop duplicates
data = data.drop_duplicates()

data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,size_number,size_model,cotton,polyester,elastane,elasterell
0,636207011,,slim_jeans,19.99,2021-08-31 20:54:18,636207,11,light_denim_blue,slim_fit,NaN,NaN,0.98,0.0,0.02,0.0
1,636207011,,slim_jeans,19.99,2021-08-31 20:54:18,636207,11,dark_denim_blue,slim_fit,NaN,NaN,0.88,0.1,0.02,0.0
2,636207011,,slim_jeans,19.99,2021-08-31 20:54:18,636207,11,dark_gray_denim,slim_fit,NaN,NaN,0.88,0.1,0.02,0.0
3,636207011,,slim_jeans,19.99,2021-08-31 20:54:18,636207,11,gray,slim_fit,NaN,NaN,0.88,0.1,0.02,0.0
4,636207011,,slim_jeans,19.99,2021-08-31 20:54:18,636207,11,black,slim_fit,NaN,NaN,0.88,0.1,0.02,0.0


# Data Load

In [12]:
query_showroom_schema = """
    CREATE TABLE showroom (
        product_id        INTEGER,
        product_category  TEXT,
        product_name      TEXT,
        product_price     REAL,
        scrapy_datetime   TEXT, 
        style_id          INTEGER, 
        color_id          INTEGER, 
        color_name        TEXT, 
        fit               TEXT,
        size_number       TEXT, 
        size_model        TEXT, 
        cotton            FLOAT, 
        polyester         FLOAT,
        elastane          FLOAT,
        elasterell        FLOAT
    )
"""

In [13]:
conn = sqlite3.connect('datasets/showroom.sqlite')
cursor = conn.execute(query_showroom_schema)
conn.commit()
conn.close()

OperationalError: table showroom already exists

In [14]:
conn = create_engine('sqlite:///showroom.sqlite', echo=False)

In [15]:
data.to_sql('showroom', con=conn, if_exists='append', index=False)

In [18]:
query = """
    SELECT * FROM showroom
"""

df = pd.read_sql_query(query, conn)
df

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,size_number,size_model,cotton,polyester,elastane,elasterell
0,636207011,,slim_jeans,19.99,2021-08-31 20:54:18.000000,636207,11,light_denim_blue,slim_fit,None,None,0.98,0.00,0.02,0.0
1,636207011,,slim_jeans,19.99,2021-08-31 20:54:18.000000,636207,11,dark_denim_blue,slim_fit,None,None,0.88,0.10,0.02,0.0
2,636207011,,slim_jeans,19.99,2021-08-31 20:54:18.000000,636207,11,dark_gray_denim,slim_fit,None,None,0.88,0.10,0.02,0.0
3,636207011,,slim_jeans,19.99,2021-08-31 20:54:18.000000,636207,11,gray,slim_fit,None,None,0.88,0.10,0.02,0.0
4,636207011,,slim_jeans,19.99,2021-08-31 20:54:18.000000,636207,11,black,slim_fit,None,None,0.88,0.10,0.02,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,720504008,men_jeans_skinny,skinny_jeans,24.99,2021-08-31 20:54:18.000000,720504,8,denim_blue,skinny_fit,None,None,0.73,0.26,0.01,0.0
219,720504008,men_jeans_skinny,skinny_jeans,24.99,2021-08-31 20:54:18.000000,720504,8,dark_blue,skinny_fit,None,None,0.73,0.26,0.01,0.0
220,720504008,men_jeans_skinny,skinny_jeans,24.99,2021-08-31 20:54:18.000000,720504,8,light_blue,skinny_fit,None,None,0.73,0.26,0.01,0.0
221,1004199002,,skinny_cropped_jeans,29.99,2021-08-31 20:54:18.000000,1004199,2,light_denim_blue,skinny_fit,187,None,0.99,0.00,0.01,0.0
